# SWOT L3 Global Mosaic - Cycle 016

567 swaths, 2 km resolution

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path
from matplotlib.colors import Normalize

## Load files

In [ ]:
data_dir = Path('swot_l3_cycle016')
files = sorted(data_dir.glob('*.nc'))
len(files)

In [ ]:
ds = xr.open_dataset(files[0])
print(ds)
ds.close()

## Plot global

In [ ]:
fig = plt.figure(figsize=(28, 14))
ax = plt.subplot(1, 1, 1, projection=ccrs.Robinson())
norm = Normalize(vmin=-0.4, vmax=0.4)
cmap = plt.cm.RdBu_r

for i, file in enumerate(files, 1):
    if i % 100 == 0:
        print(f"{i}/{len(files)}")
    
    ds = xr.open_dataset(file)
    im = ax.pcolormesh(ds.longitude.values, ds.latitude.values, ds.ssha_filtered.values,
                      transform=ccrs.PlateCarree(), cmap=cmap, norm=norm, 
                      shading='auto', rasterized=True)
    ds.close()

ax.add_feature(cfeature.LAND, facecolor='lightgray', edgecolor='none')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.gridlines(linewidth=0.5, alpha=0.5)

cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.7)
cbar.set_label('SSHA (m)', fontsize=12)

ax.set_title(f'SWOT L3 Cycle 016 - {len(files)} Swaths', fontsize=14)
plt.tight_layout()
plt.savefig('swot_l3_global.png', dpi=150, bbox_inches='tight')
plt.show()

## Regional plots

In [ ]:
regions = [
    {'name': 'NE Pacific', 'extent': [-180, -120, 40, 65]},
    {'name': 'West Coast', 'extent': [-135, -115, 30, 50]},
    {'name': 'Gulf Stream', 'extent': [-80, -60, 30, 45]},
    {'name': 'Caribbean', 'extent': [-90, -60, 10, 25]},
    {'name': 'Kuroshio', 'extent': [130, 150, 30, 45]}
]

fig = plt.figure(figsize=(20, 12))
norm = Normalize(vmin=-0.4, vmax=0.4)

for i, region in enumerate(regions):
    ax = fig.add_subplot(2, 3, i+1, projection=ccrs.PlateCarree())
    
    for file in files:
        ds = xr.open_dataset(file)
        lon, lat = ds.longitude.values, ds.latitude.values
        lon_min, lon_max, lat_min, lat_max = region['extent']
        
        if (lon.min() <= lon_max and lon.max() >= lon_min and
            lat.min() <= lat_max and lat.max() >= lat_min):
            im = ax.pcolormesh(lon, lat, ds.ssha_filtered.values,
                             transform=ccrs.PlateCarree(), cmap=plt.cm.RdBu_r,
                             norm=norm, shading='auto', rasterized=True)
        ds.close()
    
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_extent(region['extent'], crs=ccrs.PlateCarree())
    ax.set_title(region['name'], fontsize=12)

plt.tight_layout()
plt.savefig('swot_l3_regional.png', dpi=150)
plt.show()

## Extract regional subset

In [ ]:
# Example: Extract NE Pacific region
lon_min, lon_max = -180, -120
lat_min, lat_max = 40, 65

# Collect all swaths in region
regional_data = []
for file in files:
    ds = xr.open_dataset(file)
    lon = ds.longitude.values
    lat = ds.latitude.values
    
    if (lon.min() <= lon_max and lon.max() >= lon_min and
        lat.min() <= lat_max and lat.max() >= lat_min):
        regional_data.append(ds)
    else:
        ds.close()

# Save first swath as example (or combine as needed)
if regional_data:
    regional_data[0].to_netcdf('swot_l3_ne_pacific_sample.nc')
    print(f"Saved sample to swot_l3_ne_pacific_sample.nc")
    for ds in regional_data:
        ds.close()